# QuMADA tutorial 

## Enter Metadata and handle metadata

### Situation

We got a new device, we want to do a measurement on. These are the steps to enter the metadata

1. Authenticate to the Metadatabase
2. Add the device to the Metadatabase
3. Create metadata for the measurement
4. Setup the measurement
5. Run the measurement

**Remarks:**

- During steps 4 and 5, more metadata is gathered with QuMADA.

# Authenticate to the Metadatabase

In [ ]:
from qtools_metadata.db import DBConnection
connection = DBConnection()
connection.set_as_default()
connection.authenticate()

In [ ]:
connection.token.access_token

### Add device to Metadatabase

Here, we create new metadata for the device, incorporate existing layouts, factories, etc. and save the metadata to the DB.

We can use either the Web UI or create and save the metadata in code.

In [ ]:
from qtools_metadata.device import Device

# Create or load Device using the Web UI
# device = Device.create_with_webui()
device = Device.load_with_webui()
device

Alternative ways to load or create Metadata:

In [ ]:
device = Device(
    name = "my_device",
    description = "this is only a testdevice",
)

device.save()   # Save as new object to DB
device.update() # Update object on DB

In [ ]:
device = Device.get_by_id(10)
device

### Create metadata for the measurement

To create a complete Metadata set for a measurement, the same functions are used.

In [ ]:
from qtools_metadata.measurement import Measurement
from qtools_metadata.metadata import Metadata

measurement = Measurement.load_with_webui()
metadata = Metadata(measurement=measurement)

In [ ]:
measurement

### Setup the measurement

In [ ]:
import threading

import yaml
from qcodes.station import Station

from qumada.instrument.buffered_instruments import BufferedDummyDMM as DummyDmm
from qumada.instrument.buffers.buffer import map_buffers
from qumada.instrument.custom_drivers.Dummies.dummy_dac import DummyDac
from qumada.instrument.mapping import DUMMY_DMM_MAPPING, add_mapping_to_instrument, map_terminals_gui
from qumada.instrument.mapping.Dummies.DummyDac import DummyDacMapping
from qumada.measurement.scripts import (
    Generic_1D_parallel_asymm_Sweep,
    Generic_1D_parallel_Sweep,
    Generic_1D_Sweep,
    Generic_1D_Sweep_buffered,
    Generic_nD_Sweep,
    Timetrace,
)
from qumada.utils.generate_sweeps import generate_sweep, replace_parameter_settings
from qumada.utils.GUI import open_web_gui
from qumada.utils.load_from_sqlite_db import load_db, pick_measurement
from qumada.utils.ramp_parameter import *

trigger = threading.Event()

# Setup qcodes station
station = Station()


In [ ]:
# The dummy instruments have a trigger_event attribute as replacement for
# the trigger inputs of real instruments.

dmm = DummyDmm("dmm", trigger_event=trigger)
add_mapping_to_instrument(dmm, path=DUMMY_DMM_MAPPING)
station.add_component(dmm)

dac = DummyDac("dac", trigger_event=trigger)
add_mapping_to_instrument(dac, mapping=DummyDacMapping())
station.add_component(dac)

In [ ]:
load_db()

In [ ]:
# Setup measurement
buffer_settings = {
    # "trigger_threshold": 0.005,
    # "trigger_mode": "digital",
    "sampling_rate": 10,
    "duration": 5,
    "burst_duration": 5,
    "delay": 0,
}

parameters = {
    "dmm": {"voltage": {"type": "gettable"}},
    "dac": {
        "voltage": {
            "type": "dynamic",
            "setpoints": [0, 5],
        }
    },
}

In [ ]:
script = Generic_1D_Sweep_buffered()
script.setup(
    parameters,
    metadata,
    buffer_settings=buffer_settings,
    trigger_type="hardware",
    trigger_start=trigger.set,
    trigger_reset=trigger.clear,
)

In [ ]:
map_terminals_gui(station.components, script.gate_parameters, metadata=metadata)
map_buffers(station.components, script.properties, script.gate_parameters)

### Run the measurement

Metadata is saved automatically to the DB.

In [ ]:
# Run measurement
script.run(insert_metadata_into_db=True)

### Access Measurements metadata

In [ ]:
from qtools_metadata.measurement import Measurement
measurements = [meas for meas in Measurement.get_all() if meas.user == "dg" and meas.measurement_type and meas.measurement_type.name == "1D Pinchoff"]
datasets = [meas.measurement_data for meas in measurements if meas.measurement_data]
datasets

In [ ]:
Measurement.load_with_webui()